# STEW Network

I have to admit this is a bit of a mystery to me.  The hackathon task was dealing with the STEW geodatabases, not the network.  So, for starters, I have some basic questions:

  1.  According to the documentation, PopID's are the unique id.  I know the PopIDs in the geodata are not unique, are they unique in the network data?  How are they used to link with the network data?
  2.  In the network data set, elements (i.e. the `nodes`) are the organizations.  It is quite a bit larger than those in the geodata.
  3.  We have two geodata files (turfs and office locations) with 



I am going to see if I can use the geographies, and primary solution types, to `link`

# PopID's

In [ ]:
turfs_gdf = gpd.read_parquet('../data/processed/turfs.parq')
office_locations_gdf = gpd.read_parquet('../data/processed/office-locations.parq')

print(f"turfs: {len(turfs_gdf)}  office locations: {len(office_locations_gdf)}")

I am going to do some set hacking on the PopID's for starters.

**Note:** Turning these into sets `removes` the dups.  Well it doesn't remove anything, it is a property of sets!

In [ ]:
#3201/1149

#turfs_gdf.query(f"PopID == 1149")

In [ ]:
turfs_popids = set(turfs_gdf.PopID)
office_locations_popids = set(office_locations_gdf)

OK.  Goofy test #1 passes.

In [ ]:
turfs_popids.issubset(office_locations_popids)

In [ ]:
turfs_not_in_office_locations_popids = turfs_popids.difference(office_locations_popids)

len(turfs_not_in_office_locations_popids)

In [ ]:
in_both_popids = turfs_popids.union(office_locations_popids)

len(in_both_popids)

So, this is the set of PopID's in both geodata files (turfs and office locations).

Now we can look at the network data.  There are two different data frames:

  1.  elements - The organizations in the network (think nodes)
  2.  connections  - Data frame with from/to representation

In [ ]:
elements_df = pd.read_excel('../data/raw/NYC_STEWMAP_2017_Networks_Version2_Public.xlsx', sheet_name=0)
connections_df = pd.read_excel('../data/raw/NYC_STEWMAP_2017_Networks_Version2_Public.xlsx', sheet_name=2)

Now, for each of these data frames:

  1.  Examine the contents
  2.  Use set hacking on PopID's (PopID, Respondent PopID, and PopID _ALTER) and turfs/office locations
  3.  `Understand` the network structure
  4.  Decide what to *keep* 
  5.  Save to parquet files  

In [ ]:
elements_df.info()

**Notes:** 
  1.  This is indexed by PopID.  The other two are not used.
  2.  PopID is a float (will change to Int64).
  3.  There are a couple of fields to look at (indegree and metrics::last).
  4.  At face value, there's enough content to save the whole data frame.
  5.  How does the elements_df relate to the STEW geodata?

In [ ]:
elements_df['PopID'] = elements_df['PopID'].astype('Int64')

In [ ]:
elements_df.indegree.value_counts()

In [ ]:
elements_df['metrics::last'].value_counts()

At first glance they seem to be the same?  I have no idea what this means.

Are elements PopID's unique?

In [ ]:
elements_df['PopID'].is_unique

I bet it's because there are null values?  Let's see.

In [ ]:
elements_df['PopID'].isnull().sum()

I want to use the PopID to join data so I'm just using the ones that have a non-null value.

In [ ]:
elements_df.dropna(subset=['PopID'])['PopID'].is_unique

Ok.  If it has a PopID then the PopID is unique. We can work with this.j

In [ ]:
elements_popids = set(elements_df.dropna(subset=['PopID'])['PopID'])

At this point I have sets with popids from STEW map (points and polys) and elements data frame.

**Thoughts:**
   -  Are all the turfs also in office locations?
   -  I would think a union of turfs and office locations gives me all the the PopID's that filled out the questionaire?
   -  That is the set that was sent to network questionaire?
   -  I currently think the turfs are important because I'm doing spatial overlays, joins, etc. as a driver?

I am going to start by looking at the STEW-MAP data.  Are all the turfs PopID's in office locations?

In [ ]:
turfs_popids.issubset(office_locations_popids)

So, which ones are in turfs but not office locations?

In [ ]:
turfs_not_in_office_popids = turfs_popids.difference(office_locations_popids)
len(turfs_not_in_office_locations_popids)

So 8 of the turfs are not in office locations.  Sort of curious.

In [ ]:
turfs_gdf.query(f"PopID in @turfs_not_in_office_popids")['OrgName']

So these are the 8 organizations in turfs but not in office locations.  Given the duplicity of content in the two data frames, I'm not sure this matters to me?

I guess what is of interest is this same question, relating turfs and elements.

**Note:** First a slight digression.

In [ ]:
office_no_turf_popids = office_locations_popids.difference(turfs_popids)
print(f"{len(office_no_turf_popids)} office locations do not have an associated turf.")

Back to the main subject line - relating turfs geodataframe and the elements dataframe.

In [ ]:
turfs_popids.issubset(elements_popids)

So turfs is not a subset of elements, i.e. not all the turfs are in elements.

In [ ]:
turfs_not_in_elements_popids = turfs_popids.difference(elements_popids)
print(f"{len(turfs_not_in_elements_popids)} turfs are not in elements data.")

turfs_in_elements_popids = turfs_popids.intersection(elements_popids)

print(f"But {len(turfs_in_elements_popids)} are in both turfs and elements.")

This is the set with turfs PopID's that are in the elements data set.

I think this is what I'll want in later notebooks.

**Note:** Of course we'll retain all the organizations in elements since they are included in the social network.

Now let's look at the connections dataframe.

In [ ]:
connections_df.info()

First thing I see is there are lot's of null values (i.e. no information).

In [ ]:
connections_df['Version'].value_counts()

In [ ]:
connections_df['NYC_region'].value_counts()

I'm going to change the dtype of PopID _ALTER from a string to a number.  I'm using Int64 since it will deal with the NaN's.

When I do this the first time, there are five rows with a PopID _ALTER of `GENERAL`.  That doesn't work so I delected them.

In [ ]:
connections_df['PopID _ALTER'] = connections_df['PopID _ALTER'].astype('Int64')

Based on this (quick) analysis the columns I am going to `keep` are: 

  -  From: str
  -  To: Str
  -  PopID __ALTER: Int64
  -  Respondent PopID: int64
  
Based on correspondence with Michelle and Lindsay my model is [From, Respondent PopID] and [To, PopID _ALTER] are the way to think about this.

In [ ]:
connections_df = connections_df[['From', 'Respondent PopID', 'To', 'PopID _ALTER']]

In [ ]:
connections_df.info()

In [ ]:
connections_popids = set(connections_df.dropna(subset=['Respondent PopID'])['Respondent PopID'])
print(f"There are {len(connections_popids)} unique Respondent PopID's in a data set of size {len(connections_df)}")

Finally, for the PopID analysis, I want to undertand the relationship between turfs (PopIDs), elements (PopIDs) and connections (Respondent PopIDs).

We already have the set turfs_in_elements_popids to use.

In [ ]:
respondent_not_in_popids = turfs_in_elements_popids.difference(connections_popids)
respondents_in_all_popids = turfs_in_elements_popids.intersection(connections_popids)

print(f"{len(respondent_not_in_popids)} are in connections, but not turfs.")
print(f"On the other hand {len(respondents_in_all_popids)} organizations are in all (turfs, elements, and connections (i.e. respondent))")

So I'm good with this.  At least I think I understand the structure.

I am going to save these two files (as parquet files) into processed/SN for now.  I really need to build a network model/data structure but...

In [ ]:
elements_df.to_parquet('../data/processed/SN/elements.parq')
connections_df.to_parquet('../data/processed/SN/connections.parq')

Based on this analyis, the PopIDs that have good social network information is one of respondents_in_all_popids.  It might help to have this list laying around so I'm going to pickle it for later use.

In [ ]:
import pickle
pickle.dump(list(respondents_in_all_popids), open( "../data/processed/db/popids.p", "wb" ) )

When you want it back:

```python
favorite_color = pickle.load( open( "save.p", "rb" ) )
```

In [ ]:
with open('../data/processed/db/popids2.p', 'wb') as f:
    pickle.dump(list(respondents_in_all_popids), f)

At this point I have the data I want for later processing.  It is still packaged as dataframes, so it will be very `mechanical` to use.  I will revisit the db/triple store/code ... approach when I have some time.

I am curious about python packages for graph viz/analysis.  I really don't have time right now, but I'll try a couple:

In [ ]:
G = nx.Graph()

In [ ]:
connections_df.columns

In [ ]:
G = nx.from_pandas_edgelist(connections_df[:100], 'From', 'To')

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(15, 12))
nx.draw_shell(G, with_labels=True)

Good enough for starters.  Other packages to explore include igraph, and scikit-network.  Another day!